<a href="https://colab.research.google.com/github/Nikolay1221/AvitoMobileDataExplorer/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22nlp_sentiment_simple_vectorization_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Определение тональности отзывов на банки с помощью классических алгоритмов машинного обучения

Используем логистическую регрессию и мешок слов.



In [1]:
!pip install pymorphy3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 50.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import pymorphy3
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from collections import Counter

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Константы

In [4]:
max_words = 10000
random_state = 42

## Загружаем и готовим набор данных

In [7]:
!wget http://46.148.229.150:8080/evaluated_data23.csv

--2023-11-01 07:51:05--  http://46.148.229.150:8080/evaluated_data23.csv
Connecting to 46.148.229.150:8080... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1507112 (1.4M) [text/csv]
Saving to: ‘evaluated_data23.csv’

evaluated_data23.cs 100%[===================>]   1.44M  1.73MB/s    in 0.8s    

2023-11-01 07:51:07 (1.73 MB/s) - ‘evaluated_data23.csv’ saved [1507112/1507112]



In [8]:
banks = pd.read_csv('evaluated_data23.csv', sep=',', index_col='idx');

In [9]:
# Создаем экземпляр Pymorphy2 для лемматизации
morph = pymorphy3.MorphAnalyzer()

def preprocess(text, stop_words, punctuation_marks, morph):
    # Преобразовать 'text' в строку, если он не является строкой
    if not isinstance(text, str):
        text = str(text)

    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return preprocessed_text


In [10]:
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...', '«', '»', ';', '–', '--']
stop_words = stopwords.words("russian")
morph = pymorphy3.MorphAnalyzer()

In [11]:
banks['Preprocessed_texts'] = banks.apply(lambda row: preprocess(row['Text'], punctuation_marks, stop_words, morph), axis=1)

In [12]:
banks

,Score,Text,Preprocessed_texts
idx,,,
0.0,Negative,Состояние телефона отличное. * из минусов мелк...,"[состояние, телефон, отличный, *, минус, мелки..."
1.0,Negative,IPhone 15 pro. Актуальная стоимость на 21.10.2...,"[iphone, 15, pro, актуальный, стоимость, 21.10..."
2.0,Negative,Описание: новый apple iPhone 14 128gb blue. Ве...,"[описание, новый, apple, iphone, 14, 128gb, bl..."
3.0,Negative,Описание: новый samsung galaxy s22 8 256gb bla...,"[описание, новый, samsung, galaxy, s22, 8, 256..."
4.0,Positive,Есть зарядное уст. Телефон норм. Работает.,"[зарядный, уста, телефон, норма, работать]"
...,...,...,...
4619.0,Negative,Новые! Оригинальные! Запечатанные! Чек гаранти...,"[новый, оригинальный, запечатать, чек, гаранти..."
4620.0,Positive,"В хорошем состоянии, есть защитное стекло и че...","[хороший, состояние, защитный, стекло, чехол, ..."
4621.0,Negative,"Продам айфон 6s в отличном состояние, носился ...","[продать, айфон, 6s, отличный, состояние, носи..."


Считаем частоту слов во всех отзывах

In [13]:
words = Counter()

In [14]:
for txt in banks['Preprocessed_texts']:
    words.update(txt)

Создаем словарь, упорядоченный по частоте

В словаре будем использовать 2 специальных кода:
- Код заполнитель: 0
- Неизвестное слово: 1

Нумерация слов в словаре начинается с 2.

In [15]:
# Словарь, отображающий слова в коды
word_to_index = dict()
# Словарь, отображающий коды в слова
index_to_word = dict()

Создаем словари

In [16]:
for i, word in enumerate(words.most_common(max_words - 2)):
    word_to_index[word[0]] = i + 2
    index_to_word[i + 2] = word[0]

Функция для преобразования списка слов в список кодов

In [17]:
def text_to_sequence(txt, word_to_index):
    seq = []
    for word in txt:
        index = word_to_index.get(word, 1) # 1 означает неизвестное слово
        # Неизвестные слова не добавляем в выходную последовательность
        if index != 1:
            seq.append(index)
    return seq

Преобразуем все тексты в последовательность кодов слов

In [18]:
banks['Sequences'] = banks.apply(lambda row: text_to_sequence(row['Preprocessed_texts'], word_to_index), axis=1)

In [19]:
banks

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
0.0,Negative,Состояние телефона отличное. * из минусов мелк...,"[состояние, телефон, отличный, *, минус, мелки...","[2, 3, 6, 417, 346, 366, 45, 14, 3, 7, 8, 172,..."
1.0,Negative,IPhone 15 pro. Актуальная стоимость на 21.10.2...,"[iphone, 15, pro, актуальный, стоимость, 21.10...","[11, 144, 35, 265, 347, 1530, 224, 251, 31, 11..."
2.0,Negative,Описание: новый apple iPhone 14 128gb blue. Ве...,"[описание, новый, apple, iphone, 14, 128gb, bl...","[394, 7, 62, 11, 93, 80, 212, 160, 73, 395, 20..."
3.0,Negative,Описание: новый samsung galaxy s22 8 256gb bla...,"[описание, новый, samsung, galaxy, s22, 8, 256...","[394, 7, 81, 118, 974, 101, 112, 163, 241, 20,..."
4.0,Positive,Есть зарядное уст. Телефон норм. Работает.,"[зарядный, уста, телефон, норма, работать]","[24, 1038, 3, 818, 10]"
...,...,...,...,...
4619.0,Negative,Новые! Оригинальные! Запечатанные! Чек гаранти...,"[новый, оригинальный, запечатать, чек, гаранти...","[7, 43, 103, 44, 20, 60, 8674, 1951, 60, 610, ..."
4620.0,Positive,"В хорошем состоянии, есть защитное стекло и че...","[хороший, состояние, защитный, стекло, чехол, ...","[21, 2, 13, 8, 5, 3, 282, 8677]"
4621.0,Negative,"Продам айфон 6s в отличном состояние, носился ...","[продать, айфон, 6s, отличный, состояние, носи...","[27, 52, 477, 6, 2, 127, 13, 8, 273, 5, 4, 6, ..."


## Готовим данные для обучения

Преобразуем текстовые метки классов в числовые

In [20]:
mapping = {'Negative': 0, 'Positive': 1}

In [21]:
banks.replace({'Score': mapping}, inplace=True)

In [22]:
banks

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
0.0,0,Состояние телефона отличное. * из минусов мелк...,"[состояние, телефон, отличный, *, минус, мелки...","[2, 3, 6, 417, 346, 366, 45, 14, 3, 7, 8, 172,..."
1.0,0,IPhone 15 pro. Актуальная стоимость на 21.10.2...,"[iphone, 15, pro, актуальный, стоимость, 21.10...","[11, 144, 35, 265, 347, 1530, 224, 251, 31, 11..."
2.0,0,Описание: новый apple iPhone 14 128gb blue. Ве...,"[описание, новый, apple, iphone, 14, 128gb, bl...","[394, 7, 62, 11, 93, 80, 212, 160, 73, 395, 20..."
3.0,0,Описание: новый samsung galaxy s22 8 256gb bla...,"[описание, новый, samsung, galaxy, s22, 8, 256...","[394, 7, 81, 118, 974, 101, 112, 163, 241, 20,..."
4.0,1,Есть зарядное уст. Телефон норм. Работает.,"[зарядный, уста, телефон, норма, работать]","[24, 1038, 3, 818, 10]"
...,...,...,...,...
4619.0,0,Новые! Оригинальные! Запечатанные! Чек гаранти...,"[новый, оригинальный, запечатать, чек, гаранти...","[7, 43, 103, 44, 20, 60, 8674, 1951, 60, 610, ..."
4620.0,1,"В хорошем состоянии, есть защитное стекло и че...","[хороший, состояние, защитный, стекло, чехол, ...","[21, 2, 13, 8, 5, 3, 282, 8677]"
4621.0,0,"Продам айфон 6s в отличном состояние, носился ...","[продать, айфон, 6s, отличный, состояние, носи...","[27, 52, 477, 6, 2, 127, 13, 8, 273, 5, 4, 6, ..."


### Выделяем данные для обучения и тестирования

In [23]:
train, test = train_test_split(banks, test_size=0.2)

In [24]:
train

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
381.0,1,"В хорошем состоянии, пользовалась в чехле и с ...","[хороший, состояние, пользоваться, чехол, защи...","[21, 2, 86, 5, 13, 8, 15, 4]"
4418.0,0,Айфон х 64 гб всё работает на экране линия не ...,"[айфон, х, 64, гб, всё, работать, экран, линия...","[52, 406, 121, 37, 84, 10, 14, 3131, 475, 83, ..."
1608.0,0,"Не работает, модель не знаю какая.","[работать, модель, знать]","[10, 77, 388]"
4289.0,0,"Телефон в хорошем состоянии, щади небольшая тр...","[телефон, хороший, состояние, щадить, небольшо...","[3, 21, 2, 3645, 98, 95, 54, 172, 34, 133]"
2741.0,1,Xiaomi Redmi 9A новый. Упаковка не вскрыта. Од...,"[xiaomi, redmi, 9a, новый, упаковка, вскрыть, ...","[106, 130, 829, 7, 188, 717, 151, 149, 1806, 2..."
...,...,...,...,...
2310.0,0,Мобильный телефон Zte Blade A5 2020 2/32 GB Ар...,"[мобильный, телефон, zte, blade, a5, 2020, 2/3...","[180, 3, 886, 1011, 1960, 534, 1650, 69, 6236,..."
2088.0,0,Бесплатная доставка в день заказа в пределах М...,"[бесплатный, доставка, день, заказ, предел, мк...","[132, 60, 56, 150, 218, 231, 11, 71, 226, 82, ..."
3514.0,1,В отличном состоянии. Коробка. Отвязан от всех...,"[отличный, состояние, коробка, отвязать, аккау...","[6, 2, 9, 40, 46, 5, 22, 19, 26, 42]"


In [25]:
test

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
1973.0,1,"Память 3/32, слот 2 симки, экран 5,5"" Дюймов, ...","[память, 3/32, слот, 2, симка, экран, 5,5, '',...","[30, 747, 504, 57, 733, 14, 3252, 142, 499, 23..."
375.0,0,Продам нерабочие бу телефоны на запчасти: черн...,"[продать, нерабочий, бу, телефон, запчасть, чё...","[27, 1003, 542, 3, 97, 102, 1085, 4068, 4069, ..."
805.0,0,Телефон в хорошем техническом состоянии. 6ГБ о...,"[телефон, хороший, технический, состояние, 6гб...","[3, 21, 386, 2, 1815, 126, 30, 29, 549, 119, 1..."
2772.0,0,"Новый iPhone 13, Green, 128GB. Рассрочка 0-0-2...","[новый, iphone, 13, green, 128gb, рассрочка, 0...","[7, 11, 71, 348, 80, 143, 1729, 339, 47, 67, 3..."
10.0,1,Отличный телефон. Не тормозит не глючит. Носил...,"[отличный, телефон, тормозить, глючить, носить...","[6, 3, 728, 612, 127, 5, 677, 284, 15, 4, 24, ..."
...,...,...,...,...
2439.0,0,Продам или обменяю).,"[продать, обменять]","[27, 258]"
3560.0,0,"Шустрый, красивый на все случаи жизни. Полный ...","[шустрый, красивый, случай, жизнь, полный, ком...","[625, 322, 1116, 944, 15, 4, 2638, 95, 14, 141..."
1479.0,0,Друзья читаем до конца там ответы на многие во...,"[друг, читаемый, конец, ответ, многие, вопрос,...","[502, 1946, 1705, 2354, 2186, 140, 111, 31, 31..."


from matplotlib import pyplot as plt
test['Score'].plot(kind='hist', bins=20, title='Score')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
test['Score'].plot(kind='line', figsize=(8, 4), title='Score')
plt.gca().spines[['top', 'right']].set_visible(False)

### Разделяем метки классов и данные для обучения

Данные для обучения

In [26]:
x_train_seq = train['Sequences']
y_train = train['Score']

In [27]:
x_train_seq

idx
381.0                          [21, 2, 86, 5, 13, 8, 15, 4]
4418.0    [52, 406, 121, 37, 84, 10, 14, 3131, 475, 83, ...
1608.0                                        [10, 77, 388]
4289.0           [3, 21, 2, 3645, 98, 95, 54, 172, 34, 133]
2741.0    [106, 130, 829, 7, 188, 717, 151, 149, 1806, 2...
                                ...                        
2310.0    [180, 3, 886, 1011, 1960, 534, 1650, 69, 6236,...
2088.0    [132, 60, 56, 150, 218, 231, 11, 71, 226, 82, ...
3514.0                 [6, 2, 9, 40, 46, 5, 22, 19, 26, 42]
3455.0    [11, 51, 35, 90, 80, 1742, 212, 36, 20, 143, 2...
3536.0    [3546, 11, 72, 35, 161, 348, 558, 279, 372, 7,...
Name: Sequences, Length: 3699, dtype: object

In [28]:
y_train

idx
381.0     1
4418.0    0
1608.0    0
4289.0    0
2741.0    1
         ..
2310.0    0
2088.0    0
3514.0    1
3455.0    0
3536.0    0
Name: Score, Length: 3699, dtype: int64

Данные для тестирования

In [29]:
x_test_seq = test['Sequences']
y_test = test['Score']

In [30]:
x_test_seq

idx
1973.0    [30, 747, 504, 57, 733, 14, 3252, 142, 499, 23...
375.0     [27, 1003, 542, 3, 97, 102, 1085, 4068, 4069, ...
805.0     [3, 21, 386, 2, 1815, 126, 30, 29, 549, 119, 1...
2772.0    [7, 11, 71, 348, 80, 143, 1729, 339, 47, 67, 3...
10.0      [6, 3, 728, 612, 127, 5, 677, 284, 15, 4, 24, ...
                                ...                        
2439.0                                            [27, 258]
3560.0    [625, 322, 1116, 944, 15, 4, 2638, 95, 14, 141...
1479.0    [502, 1946, 1705, 2354, 2186, 140, 111, 31, 31...
3772.0    [27, 11, 51, 80, 407, 2, 178, 15, 4, 7660, 552...
231.0                   [23, 1282, 3891, 48, 17, 2, 28, 34]
Name: Sequences, Length: 925, dtype: object

In [31]:
y_test

idx
1973.0    1
375.0     0
805.0     0
2772.0    0
10.0      1
         ..
2439.0    0
3560.0    0
1479.0    0
3772.0    1
231.0     1
Name: Score, Length: 925, dtype: int64

## Создаем мешок слов

In [32]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for index in sequence:
            results[i, index] += 1.
    return results

In [33]:
x_train = vectorize_sequences(x_train_seq, max_words)

In [34]:
x_test = vectorize_sequences(x_test_seq, max_words)

In [35]:
x_train[0][:100]

array([0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [36]:
len(x_train[0])

10000

## Создаем модель машинного обучения

In [37]:
lr = LogisticRegression(random_state=random_state, max_iter=500)

## Обучаем модель машинного обучения

In [38]:
lr.fit(x_train, y_train)

LogisticRegression(max_iter=500, random_state=42)

## Оцениваем качество обучения на тестовом наборе данных

Определяем долю правильных ответов (accuracy) на тестовом наборе данных

In [39]:
lr.score(x_test, y_test)

0.84

## Применяем модель для определения тональности отзыва на банк

**Позитивный отзыв**

In [40]:
positive_text = """Продам оригинальный чехол, iPhone 12 Pro Max Silicone Case magsafe Deep черный."""

Подготовка текста к обработке

In [41]:
positive_preprocessed_text = preprocess(positive_text, stop_words, punctuation_marks, morph)

In [42]:
positive_preprocessed_text

['продать',
 'оригинальный',
 'чехол',
 'iphone',
 '12',
 'pro',
 'max',
 'silicone',
 'case',
 'magsafe',
 'deep',
 'чёрный']

In [43]:
positive_seq = text_to_sequence(positive_preprocessed_text, word_to_index)

In [44]:
positive_seq

[27, 43, 5, 11, 51, 35, 90, 4114, 1829, 1406, 1199, 102]

In [45]:
positive_bow = vectorize_sequences([positive_seq], max_words)

In [46]:
positive_bow[0][0:100]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Выполняем распознавание

In [47]:
result = lr.predict(positive_bow)

In [48]:
result

array([1])

**Негативный отзыв**

In [61]:
negative_text = """Продаю телефон. У нас только новинки."""

In [62]:
negative_preprocessed_text = preprocess(negative_text, stop_words, punctuation_marks, morph)
negative_seq = text_to_sequence(negative_preprocessed_text, word_to_index)
negative_bow = vectorize_sequences([negative_seq], max_words)

In [63]:
negative_bow[0][0:100]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [64]:
result = lr.predict(negative_bow)

In [65]:
result

array([1])

In [66]:
result = lr.predict_proba(negative_bow)

In [67]:
result

array([[0.2576036, 0.7423964]])